# Домащнее задание 2

## Бейзлайны, Детерминированные алгоритмы

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [64]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [65]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [133]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code

    recs = np.random.choice(items_weights.item_id,
                            p=items_weights.weight, 
                            size=n,
                            replace=False)

    return recs.tolist()

In [67]:
%%time

# your_code

# Товары с рейтингом популярности.
items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()

# Логарифм от объема продаж. Для товаров с суммой покупки меньше 1$ считаем вероятность купить этот товар равной нулю.
items_weights['log_sales_value'] = items_weights.sales_value.apply(lambda x: 0 if x < 1 else np.log(x))

# Сумма log_sales_value по всем товарам.
log_sales_value_sum = items_weights.log_sales_value.sum()

# Расчет веса товара по популярности.
items_weights['weight'] = items_weights.log_sales_value.apply(lambda x: x / log_sales_value_sum)

items_weights.head() 

CPU times: user 156 ms, sys: 173 µs, total: 156 ms
Wall time: 155 ms


,item_id,sales_value,log_sales_value,weight
0,25671,20.94,3.041661,0.000013
1,26081,0.99,0.000000,0.000000
2,26093,1.59,0.463734,0.000002
3,26190,1.54,0.431782,0.000002
4,26355,1.98,0.683097,0.000003


Проверим правильность расчета весов.

In [68]:
popular.weight.sum()

1.0

Сформируем датафрейм для сохранения всех результатов.

In [69]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Добавим взвешенные случайные рекомендации к итоговому датафрейму.

In [70]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 1.64 s, sys: 0 ns, total: 1.64 s
Wall time: 1.64 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

Здесь для ускорения процесса и для сохранения различных алгоритмов в истории (в git репозитории), скопируем методы из методички и проведём рассчет датафрейма `results`.  

### Random recommendation

In [71]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [72]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 2.49 s, sys: 0 ns, total: 2.49 s
Wall time: 2.48 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]"


### Popularity-based recommendation

In [73]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [74]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 69.8 ms, sys: 0 ns, total: 69.8 ms
Wall time: 68.8 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Item-Item Recommender / ItemKNN

In [75]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [76]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [77]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
user_item_matrix.shape

(2499, 5001)

In [79]:
# Разреженность матрицы.
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [80]:
# Словари для преобразования id

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [92]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)  # На вход item-user matrix

100%|██████████| 5001/5001 [00:00<00:00, 18244.24it/s]CPU times: user 1.24 s, sys: 0 ns, total: 1.24 s
Wall time: 420 ms



In [82]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 47.5 ms, sys: 0 ns, total: 47.5 ms
Wall time: 46.4 ms


In [83]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]"


### CosineRecommender

In [84]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

100%|██████████| 5001/5001 [00:00<00:00, 21001.95it/s]CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 396 ms



In [85]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 59 ms, sys: 3.48 ms, total: 62.5 ms
Wall time: 61.7 ms


In [86]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]"


### TFIDFRecommender

In [91]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

100%|██████████| 5001/5001 [00:00<00:00, 20410.73it/s]CPU times: user 1.12 s, sys: 0 ns, total: 1.12 s
Wall time: 413 ms



In [88]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 64.5 ms, sys: 13 µs, total: 64.5 ms
Wall time: 63.8 ms


In [89]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]"


### Рекомендации среди уже купленных товаров

In [90]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

100%|██████████| 5001/5001 [00:00<00:00, 17328.01it/s]CPU times: user 1.25 s, sys: 7.12 ms, total: 1.25 s
Wall time: 433 ms



In [93]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 58.5 ms, sys: 0 ns, total: 58.5 ms
Wall time: 57.5 ms


In [94]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1352718, 7441799, 967372, 944024, 15800995]","[1017573, 6704162, 15452266, 1714184, 1104424]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1016947, 13877156, 6979364, 9420068, 9297517]","[8181378, 490831, 2066389, 920528, 10463651]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 981760, 1098066, 995242]"


Итоговый датафрейм сформирован, далее посчитаем метрику `Precision@5`.

In [ ]:
# your_code

In [95]:

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [113]:
columns = result.columns.drop(['user_id', 'actual'])
actual = result.actual
users_count = result.shape[0]

In [131]:
for column in columns:
    sum_p = 0
    precisions_by_user = [precision_at_k(actual[i], result[column][i], k=5) for i in range(users_count)]
    precisions_mean = np.mean(precisions_by_user)

    print('{:35} {:.4f} %'.format(column, precisions_mean * 100))

weighted_random_recommendation      0.0098 %
random_recommendation               0.0000 %
popular_recommendation              2.7138 %
itemitem                            0.7476 %
cosine                              0.6995 %
tfidf                               0.7484 %
own_purchases                       1.0708 %


Наилучшее качество показывает бейзлайн алгоритм рекомендации товаров по популярности. Этот алгоритм не зависит от пользователя и рекомендует всем одни и те же товары. 

Алгоритм рекомендации случайного товара показал нулевую точность, а аналогичный алгоритм с учетом весов товаров по объемам продаж дал уже не нулевой результат.  

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [ ]:
# your_code